# Introduction
Author: Yingding Wang\
Created on: 05.01.2024

This notebook file demonstrate the simple Kubeflow V2 pipeline on Kubeflow 1.8.0 manifests deployment with ml pipeline api server 2.0.5. 

## KFP v2 docs
* KFP v1 to v2 migration https://www.kubeflow.org/docs/components/pipelines/v2/migration/
* Hello world v2 https://www.kubeflow.org/docs/components/pipelines/v2/hello-world/

In [1]:
import sys
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.11.6


In [2]:
#!{sys.executable} -m pip install --upgrade --user kfp==2.4.0 kfp-kubernetes==1.0.0 kfp-server-api==2.0.5

In [3]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-kubernetes                1.0.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.5


In [4]:
from kfp.dsl.pipeline_task import PipelineTask

def set_res_limit(task: PipelineTask, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m') -> PipelineTask:
    """set the resource limit for cpu and memory, no cpu and memory requirement sofar.
    should the limit is set to small, the Task Pod would be stopped by kubernetes with OOMKilled status.
    
    Args:
        task(PipelineTask): the KFP PipelineTask which need to be set the cpu and memory limits
        cpu_limit(str): the str representation of cpu limit e.g. '1' as one cpu time, '0.5' as 1/2 cpu time
        mem_limit(str): the str representation of memory limit e.g. '500M' for 500MB RAM
        
    Return:
        (PipelineTask): the PipelineTask with the desired limitations set
    """
    # return task.set_cpu_limit('1').set_memory_limit('500M')
    return task.set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)\
            .set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)

### Simple pipeline
* hello world v2 https://www.kubeflow.org/docs/components/pipelines/v2/hello-world/

In [5]:
from dataclasses import dataclass

@dataclass
class Settings():
    # base_image='python:3.10.13-bullseye' 
    base_image='python:3.11.7-bullseye'

settings = Settings()

def gen_compiled_file_path(file_name: str, pipeline_path_dir="./compiled") -> str:
    return f"{pipeline_path_dir}/{file_name}.yaml"

In [6]:
from kfp import dsl

component_file_name = "hello_component"
component_file_path = gen_compiled_file_path(component_file_name)

@dsl.component(
    base_image=settings.base_image,
    output_component_file=component_file_path
)
def say_hello(name: str) -> str:
    hello_text = f"Hello, {name}!"
    print(hello_text)
    return hello_text

@dsl.pipeline(
    name='Helloworld pipeline',
    description='An example kfp v2 simple pipeline'
)
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    # set resource limit for the task pod 200Mi, 1CPU
    hello_task = set_res_limit(task=hello_task, mem_req="200Mi", cpu_req="1000m", mem_lim="500Mi", cpu_lim='1000m')
    return hello_task.output

my_pipeline = hello_pipeline

/tmp/ipykernel_6119/1585871676.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @dsl.component(


In [7]:
from kfp import compiler
import os

my_pipeline_file_name = "hello_pipeline_v2"
pipeline_package_path = gen_compiled_file_path(my_pipeline_file_name)

pipeline_path_dir="./compiled"
if not os.path.exists(pipeline_path_dir):
    os.makedirs(pipeline_path_dir)

compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path=pipeline_package_path
)

In [8]:
from kfp.client import Client
import warnings

'''suppress kfp v2 client FutureWarning
https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings/14463362#14463362
'''
with warnings.catch_warnings(action="ignore", category=FutureWarning):
    # kubeflow pipeline poddefault is passing the credential to the client
    client = Client()

NAMESPACE = client.get_user_namespace()
EXPERIMENT_NAME = "demo"
print(NAMESPACE)

args = {
    'recipient': 'World',
}
ENABLE_CACHING = True

kubeflow-kindfor


### Run from pipeline function

In [9]:
run = client.create_run_from_pipeline_func(
    pipeline_func = my_pipeline, 
    experiment_name = EXPERIMENT_NAME,
    namespace = NAMESPACE,
    enable_caching=ENABLE_CACHING,
    arguments=args,
)
run

RunPipelineResult(run_id=648578d1-f0cc-4ae1-b751-0938a5025dd2)

### Run from pipeline package

In [10]:
# run = client.create_run_from_pipeline_package(
#     pipeline_file=pipeline_package_path,
#     experiment_name = EXPERIMENT_NAME,
#     namespace = NAMESPACE,
#     enable_caching=ENABLE_CACHING,
#     arguments=args,
# )
# run